In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None) # For Train
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None) # For eval

In [ ]:
cath_info = cath_df.info()

In [ ]:
ecod_info = ecod_df.info()

In [ ]:
# Summarize shapes, classes, and a preview
cath_summary = {
    "shape": cath_df.shape,
    "unique_classes": cath_df[0].nunique(),
    "class_distribution": cath_df[0].value_counts(),
    "head": cath_df.head()
}

ecod_summary = {
    "shape": ecod_df.shape,
    "unique_classes": ecod_df[0].nunique(),
    "class_distribution": ecod_df[0].value_counts(),
    "head": ecod_df.head()
}

In [ ]:
print(cath_summary)

In [ ]:
print(ecod_summary)

In [ ]:
for i in range(0,2):
    print(type(cath_df.iloc[0,i]))

In [ ]:
# Plot class distribution for CATH
cath_class_counts = cath_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(cath_class_counts, bins=30, edgecolor='black')
plt.title('Class Frequency Distribution in CATH Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot class distribution for ECOD
ecod_class_counts = ecod_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(ecod_class_counts, bins=20, edgecolor='black')
plt.title('Class Frequency Distribution in ECOD Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()


### Visualizing Class Imbalance

Each protein in the dataset is associated with a structural class label (column 0), which identifies its 3D(1D in this case) shape category. Understanding how many proteins fall into each class is critical because the dataset is not balanced—some classes have many proteins, while others have very few.

To quantify this, we use `value_counts()` to compute the frequency of each class label and visualize the distribution with a histogram.

#### Why this is important:
During training, we will create pairs of proteins to determine structural similarity. If a particular class contains many proteins, it will generate significantly more pairs, which may bias the model toward frequently occurring classes. This can lead to overfitting and poor generalization, especially on underrepresented classes.

By plotting the class frequency distribution:
- We confirm whether class imbalance is present.
- We motivate the need for sampling strategies such as `WeightedRandomSampler` to correct for this imbalance during training.

This analysis is a key step in understanding the structure of the data and informing how we design the training process.


In [1]:
import sys
import numpy as np

print("Python version:", sys.version)
print("NumPy version:", np.__version__)


Python version: 3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 12:57:28) [Clang 14.0.6 ]
NumPy version: 1.26.4


In [16]:
import pandas as pd
import torch
import importlib
import train
import time

importlib.reload(train)

from train import train_model, test_model_on_ecod

In [17]:
# Load datasets
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None).dropna(axis=1)
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None).dropna(axis=1)

print(f"CATH: {cath_df.shape}, ECOD: {ecod_df.shape}")


CATH: (2685, 3923), ECOD: (761, 3923)


In [6]:
import pandas as pd
from cache_utils import cache_pairwise_data

df = pd.read_csv("./data/cath_moments.tsv", sep="\t", header=None).dropna(axis=1)
num_proteins = 200
cache_dir = "./cache/cath_"+str(num_proteins)
cache_pairwise_data(df.head(num_proteins), cache_dir=cache_dir, buffer_limit_mb=100)

Generating pairwise data from 200 proteins using 12 cores...
Flushed buffer 0 with 1 items → ./cache/cath_200/part_0.pkl
Flushed buffer 1 with 1000 items → ./cache/cath_200/part_1.pkl
Flushed buffer 2 with 1000 items → ./cache/cath_200/part_2.pkl
Flushed buffer 3 with 1000 items → ./cache/cath_200/part_3.pkl
Flushed buffer 4 with 1000 items → ./cache/cath_200/part_4.pkl
Flushed buffer 5 with 1000 items → ./cache/cath_200/part_5.pkl
Flushed buffer 6 with 1000 items → ./cache/cath_200/part_6.pkl
Flushed buffer 7 with 1000 items → ./cache/cath_200/part_7.pkl
Flushed buffer 8 with 1000 items → ./cache/cath_200/part_8.pkl
Flushed buffer 9 with 1000 items → ./cache/cath_200/part_9.pkl
Flushed buffer 10 with 1000 items → ./cache/cath_200/part_10.pkl
Flushed buffer 11 with 1000 items → ./cache/cath_200/part_11.pkl
Flushed buffer 12 with 1000 items → ./cache/cath_200/part_12.pkl
Flushed buffer 13 with 1000 items → ./cache/cath_200/part_13.pkl
Flushed buffer 14 with 1000 items → ./cache/cath_200

In [10]:
from cache_utils import load_cached_parts,load_and_merge_parts
from dataset import ProteinPairDataset

tic = time.time_ns()
# Step 1: Load and merge buffered part_*.pkl files
# features, labels = load_cached_parts("cache/cath_buffered", max_threads=16)

#Parallel Load all the parts
features, labels = load_and_merge_parts(
    cache_dir=cache_dir,
    save_path=cache_dir+"/cath_merged.pkl", 
    max_threads=16
)

tac = time.time_ns()
print("Loaded in ",(tac-tic)/(10**6),"ms")

Loading 21 parts from ./cache/cath_200 using 16 threads...


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 286.61it/s]


 Merged total pairs: 19900
 Saved merged dataset to: ./cache/cath_200/cath_merged.pkl
Loaded in  1591.912 ms


In [11]:
# Step 2: Pass them into your dataset directly
dataset = ProteinPairDataset(features=features, labels=labels)

In [13]:
print("Expected pairs:",num_proteins*(num_proteins-1)//2)

Expected pairs: 19900


In [18]:
# Optional: re-import the functions explicitly
from train import train_model, test_model_on_ecod

# # Train logistic regression
# model_logistic = train_model(cath_df, hidden_dim=None, num_epochs=5, batch_size=8)

# Train small neural net
model = train_model(features=features, labels=labels, hidden_dim=64, num_epochs=5)

Training on: cpu (CPU)
Training model (hidden_dim=64) for 5 epochs...


Epoch 1/5:   0%|          | 0/3980 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 2/5:   0%|          | 0/3980 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 5/5:   0%|          | 0/3980 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/vk/4clvfdvn29z3t_m4kf0z2y7c0000gn/T/ipykernel_42371/1393508396.py", line 8, in <module>
    model = train_model(features=features, labels=labels, hidden_dim=64, num_epochs=5)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/akshitagarwal/Akshit/UCSD/SP25/rcsb_pdb_nn/train.py", line 141, in train_model
    write.close()
    ^^^^^
NameError: name 'write' is not defined. Did you mean: 'writer'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
model = train_model(features=features, labels=labels, hidden_dim=None, num_epochs=5)